## 1. First step 
## Brochure for website

In [68]:
from pprint import pprint
from agent_tools.webscraper_tools import WebScraper
import requests
from IPython.display import Markdown, display, update_display
from openai import OpenAI
import json

In [69]:
MODEL = "gemma-3n-e4b"
API_KEY = "lm-studio"

In [70]:
link_system_prompt = f"You are provided with a list of links found on a webpage. Strictly follow the rules, Do not change anything\n"
link_system_prompt += f"You need to return all the links in JSON format as follows:\n"
link_system_prompt += """{
        links: [
            {"type": "about page", "url":"https://example.com/about"},
            {"type": "careers", "url":"https://example.com/careers"}
    ] 
}"""
    

In [71]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the webpage: {website.url}\n"
    user_prompt += "Please decide which of these are relevant web links for a brochure about company, respond with the full https URL: Do not include Terms of Service, Privacy Policy, Cookie Policy, or any other legal links such as linkedIn, youtube, X, twitter, etc. Please only include links for given URL from available URLs only\n"
    user_prompt += f"Links: (some might be relative links):\n"
    user_prompt += f"\n".join(website.get_all_links())
    return user_prompt

In [72]:
client = OpenAI(base_url="http://192.168.31.62:1234/v1", api_key=API_KEY)

In [73]:
def get_links(url):
    website = WebScraper(url)
    response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)},
        ],
    )
    result = response.choices[0].message.content
    if result is None:
        return {"links": []}
    cleaned_result = result.strip("```json").strip("```")
    print(f"[INFO] Extracted Links JSON:")
    display(Markdown(result))
    return json.loads(cleaned_result)

In [74]:
def get_all_details(url):
    result = "Landing page:\n"
    
    landing_content = WebScraper(url).get_body()
    result += landing_content if landing_content else "[Failed to fetch landing page]\n"

    links_dict = get_links(url)

    for link in links_dict['links']:
        result += f"\n\n{link['type'].capitalize()}:\n"
        link_content = WebScraper(link['url']).get_body()
        result += link_content if link_content else f"[Failed to fetch {link['url']}]\n"

    return result

In [75]:
brochure_system_prompt = "You are an assistant thaat analyzes the content of several relevant pages from copany website and creates a short brochre about the company for prospective customers, \
    investors. Respond in markdown format. Include details of comppany culture, custumers and careers/jobs if you have the information."

In [76]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here is the content from several relevant pages on their website: {url}\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:10_000] # Truncate to first 10,000 characters
    return user_prompt

In [77]:
def create_brochure(company_name, url):
    stream = client.chat.completions.create(
        model = MODEL,
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)},
        ],
        stream=True
    )
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ""
        response = response.replace("```", "").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [78]:
create_brochure("Anthropic", "https://www.anthropic.com/")

[INFO] Extracted Links JSON:


```json
{
        "links": [
            {"type": "about page", "url":"https://www.anthropic.com/"},
            {"type": "product", "url":"https://www.anthropic.com/claude"},
            {"type": "product", "url":"https://www.anthropic.com/claude-code"},
            {"type": "product", "url":"https://www.anthropic.com/max"},
            {"type": "team", "url":"https://www.anthropic.com/team"},
            {"type": "enterprise", "url":"https://www.anthropic.com/enterprise"},
            {"type": "pricing", "url":"https://www.anthropic.com/pricing"},
            {"type": "download", "url":"https://claude.ai/download"},
            {"type": "product", "url":"https://claude.ai/"},
            {"type": "news", "url":"https://www.anthropic.com/news/claude-character"},
            {"type": "api", "url":"https://www.anthropic.com/api"},
            {"type": "documentation", "url":"https://docs.anthropic.com/"},
            {"type": "pricing", "url":"https://www.anthropic.com/pricing#api"},
            {"type": "console", "url":"https://console.anthropic.com/"},
            {"type": "documentation", "url":"https://docs.anthropic.com/en/docs/welcome"},
            {"type": "solutions", "url":"https://www.anthropic.com/solutions/agents"},
            {"type": "solutions", "url":"https://www.anthropic.com/solutions/code-modernization"},
            {"type": "solutions", "url":"https://www.anthropic.com/solutions/coding"},
            {"type": "solutions", "url":"https://www.anthropic.com/solutions/customer-support"},
            {"type": "solutions", "url":"https://www.anthropic.com/solutions/education"},
            {"type": "solutions", "url":"https://www.anthropic.com/solutions/financial-services"},
            {"type": "solutions", "url":"https://www.anthropic.com/solutions/government"},
            {"type": "customers", "url":"https://www.anthropic.com/customers"},
            {"type": "research", "url":"https://www.anthropic.com/research"},
            {"type": "economic index", "url":"https://www.anthropic.com/economic-index"},
            {"type": "product", "url":"https://www.anthropic.com/claude/opus"},
            {"type": "product", "url":"https://www.anthropic.com/claude/sonnet"},
            {"type": "product", "url":"https://www.anthropic.com/claude/haiku"},
            {"type": "news", "url":"https://www.anthropic.com/news/visible-extended-thinking"},
            {"type": "transparency", "url":"https://www.anthropic.com/transparency"},
            {"type": "news", "url":"https://www.anthropic.com/news/announcing-our-updated-responsible-scaling-policy"},
            {"type": "news", "url":"https://www.anthropic.com/news/anthropic-achieves-iso-42001-certification-for-responsible-ai"},
            {"type": "engineering", "url":"https://www.anthropic.com/engineering"},
            {"type": "learn", "url":"https://www.anthropic.com/learn"},
            {"type": "company", "url":"https://www.anthropic.com/company"},
            {"type": "careers", "url":"https://www.anthropic.com/careers"},
            {"type": "events", "url":"https://www.anthropic.com/events"},
            {"type": "engineering", "url":"https://www.anthropic.com/engineering/building-effective-agents"},
            {"type": "news", "url":"https://www.anthropic.com/news"},
            {"type": "download", "url":"https://claude.ai/login"},
            {"type": "product", "url":"https://claude.ai/download"},
            {"type": "product", "url":"https://www.anthropic.com/claude"},
            {"type": "product", "url":"https://www.anthropic.com/claude-code"},
            {"type": "product", "url":"https://www.anthropic.com/max"},
            {"type": "team", "url":"https://www.anthropic.com/team"},
            {"type": "enterprise", "url":"https://www.anthropic.com/enterprise"},
            {"type": "pricing", "url":"https://www.anthropic.com/pricing"},
            {"type": "download", "url":"https://claude.ai/download"},
            {"type": "product", "url":"https://claude.ai/"},
            {"type": "news", "url":"https://www.anthropic.com/news/claude-opus-4-1"},
            {"type": "product", "url":"https://www.anthropic.com/claude/opus"},
            {"type": "product", "url":"https://www.anthropic.com/claude/sonnet"},
            {"type": "news", "url":"https://www.anthropic.com/news/core-views-on-ai-safety"},
            {"type": "news", "url":"https://www.anthropic.com/rsp-updates"},
            {"type": "learn", "url":"https://www.anthropic.com/learn"},
            {"type": "news", "url":"https://www.anthropic.com/news/1m-context"},
            {"type": "news", "url":"https://www.anthropic.com/news/claude-opus-4-1"},
            {"type": "research", "url":"https://www.anthropic.com/research/project-vend-1"},
            {"type": "research", "url":"https://www.anthropic.com/research/agentic-misalignment"},
            {"type": "news", "url":"https://www.anthropic.com/news/claude-4"},
            {"type": "research", "url":"https://www.anthropic.com/research/tracing-thoughts-language-model"},
            {"type": "economic index", "url":"https://www.anthropic.com/economic-index"},
            {"type": "jobs", "url":"https://www.anthropic.com/jobs"},
            {"type": "partners", "url":"https://www.anthropic.com/partners/mcp"},
            {"type": "partners", "url":"https://www.anthropic.com/partners/powered-by-claude"},
            {"type": "company", "url":"https://www.anthropic.com/company"},
            {"type": "careers", "url":"https://www.anthropic.com/careers"},
            {"type": "events", "url":"https://www.anthropic.com/events"},
            {"type": "news", "url":"https://www.anthropic.com/news"},
            {"type": "startups", "url":"https://www.anthropic.com/startups"},
            {"type": "status", "url":"https://status.anthropic.com/"},
            {"type": "countries", "url":"https://www.anthropic.com/supported-countries"},
            {"type": "support", "url":"https://support.anthropic.com/"}
        ]
}
```

[WARN] Failed to fetch https://claude.ai/download - Status code: 403
[WARN] Failed to fetch https://claude.ai/ - Status code: 403
[WARN] Failed to fetch https://console.anthropic.com/ - Status code: 403
[WARN] Failed to fetch https://claude.ai/login - Status code: 403
[WARN] Failed to fetch https://claude.ai/download - Status code: 403
[WARN] Failed to fetch https://claude.ai/download - Status code: 403
[WARN] Failed to fetch https://claude.ai/ - Status code: 403



# Anthropic: Building AI for a Safer Future

## About Us

Anthropic is an AI research and deployment company dedicated to building safe and beneficial artificial intelligence. Founded on the principle that powerful AI requires careful consideration of its societal impact, we prioritize human well-being in every aspect of our work – from research and policy to product design. We strive to demonstrate what responsible AI development looks like in practice. 

## Our Core Values & Approach

*   **AI Safety:**  Our core mission is to build AI that serves humanity's long-term well-being, proactively addressing potential risks and unintended consequences.
*   **Responsible Scaling:** We are committed to developing and deploying AI responsibly, with a focus on safety and societal impact. Our Responsible Scaling Policy outlines our approach.
*   **Transparency & Accountability:**  We believe in open communication about our research, policies, and product design. 
*   **Research-Driven:** Anthropic invests heavily in cutting-edge AI research to advance the field safely and effectively.

## Key Products & Technologies

### Claude

Claude is a family of large language models designed for a wide range of applications.  The latest models include:

*   **Claude Opus 4.1:** Our most intelligent model yet, offering enhanced capabilities in coding, AI agents, and handling complex tasks.
*   **Claude Sonnet 4:** A powerful model excelling at both general and specialized tasks. Notably includes a version with a 1M context window.
*   **Claude Haiku 3.5:**  A lightweight model ideal for quick and efficient tasks.

### Claude Code

Designed specifically for coding, Claude Code helps developers build better software faster. It excels at code generation, understanding, and debugging.

### API Platform

Our API platform allows developers to integrate Claude's powerful AI capabilities into their own applications and workflows.


## Solutions & Applications

Anthropic's technology is being applied across various industries:

*   **AI Agents:** Building intelligent agents that can automate complex tasks.
*   **Code Modernization:** Assisting with the modernization of existing codebases.
*   **Customer Support:** Enhancing customer service interactions.
*   **Education:** Supporting innovative learning experiences.
*  **Financial Services, Government & More**: We tailor solutions for a variety of sectors

## Careers

Anthropic is growing! If you're passionate about AI and making a positive impact on the world, we encourage you to explore career opportunities: [See open roles](https://www.anthropic.com/careers). 

##  Learn More

*   **Anthropic Academy:** A resource for learning how to build with Claude.
*   **Research:** Explore our latest research findings and publications.
* **News**: Stay updated on Anthropic's latest announcements, including model releases and policy updates.



## Contact Us

Interested in learning more about Anthropic or exploring potential partnerships?  Please [Speak with sales](https://www.anthropic.com/contact).


